In [1]:
# Run setup code
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tltk
# tltk.pos_load()
%matplotlib inline

In [2]:
# Path to the preprocessed data
best_processed_path = 'w_review_train.csv'

In [3]:
# Create a character map
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}
print(CHARS_MAP)
print(len(CHARS_MAP))

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '<': 29, '=': 30, '>': 31, '?': 32, '@': 33, 'A': 34, 'B': 35, 'C': 36, 'D': 37, 'E': 38, 'F': 39, 'G': 40, 'H': 41, 'I': 42, 'J': 43, 'K': 44, 'L': 45, 'M': 46, 'N': 47, 'O': 48, 'P': 49, 'Q': 50, 'R': 51, 'S': 52, 'T': 53, 'U': 54, 'V': 55, 'W': 56, 'X': 57, 'Y': 58, 'Z': 59, '[': 60, '\\': 61, ']': 62, '^': 63, '_': 64, 'a': 65, 'b': 66, 'c': 67, 'd': 68, 'e': 69, 'f': 70, 'g': 71, 'h': 72, 'i': 73, 'j': 74, 'k': 75, 'l': 76, 'm': 77, 'n': 78, 'o': 79, 'other': 80, 'p': 81, 'q': 82, 'r': 83, 's': 84, 't': 85, 'u': 86, 'v': 87, 'w': 88, 'x': 89, 'y': 90, 'z': 91, '}': 92, '~': 93, 'ก': 94, 'ข': 95, 'ฃ': 96, 'ค': 97, 'ฅ': 98, 'ฆ': 99, 'ง': 100, 'จ': 101, 'ฉ': 102, 'ช': 103, 'ซ': 104, 'ฌ': 105, 'ญ': 106, 'ฎ': 107, 'ฏ': 108, 'ฐ': 109, 'ฑ'

In [4]:
from keras.models import Model
from keras.layers import Embedding,Dense, Conv1D, Flatten, TimeDistributed, Dropout
from keras.layers import Input, GRU, Bidirectional
from keras.optimizers import Adam
# def get_my_best_model2():
#     input1 = Input(shape=(21,))
#     x = Embedding(178,8)(input1)
#     x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
#     x = TimeDistributed(Dense(5))(x)ฅ
#     x = Flatten()(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     out = Dense(1, activation='sigmoid')(x)
#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                  loss='binary_crossentropy',
#                  metrics=['acc'])          
#     return model

def get_my_best_model():
    input1 = Input(shape=(21,))
    x = Embedding(178,8)(input1)
    x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
    x = TimeDistributed(Dense(5))(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='binary_crossentropy',
                 metrics=['acc'])          
    return model
    

Using TensorFlow backend.


In [5]:
weight_path_my_best_model2='model_best.h5'
my_best_model2 = get_my_best_model()
my_best_model2.load_weights(weight_path_my_best_model2)
#my_best_model2.make_predict_function()
my_best_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 21, 8)             1424      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 21, 100)           4100      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 5)             505       
_________________________________________________________________
flatten_1 (Flatten)          (None, 105)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10600     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
__________

In [6]:
df = pd.read_csv(best_processed_path, sep=';', header=None)

In [7]:
# df

In [8]:
def create_n_gram_df(df, n_pad):
  """
  Given an input dataframe, create a feature dataframe of shifted characters
  Input:
  df: timeseries of size (N)
  n_pad: the number of context. For a given character at position [idx],
    character at position [idx-n_pad/2 : idx+n_pad/2] will be used 
    as features for that character.
  
  Output:
  dataframe of size (N * n_pad) which each row contains the character, 
    n_pad_2 characters to the left, and n_pad_2 characters to the right
    of that character.
  """
  n_pad_2 = int((n_pad - 1)/2)
  for i in range(n_pad_2):
      df['char-{}'.format(i+1)] = df[0].shift(i + 1)
      df['char{}'.format(i+1)] = df[0].shift(-i - 1)
  return df[n_pad_2: -n_pad_2]

In [9]:
word_test = [x for x in df[0][1]]
print(''.join(word_test))

อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา ตัวร้านตั้งอยู่ที่ถนนพุทธมณฑลสาย 3 ไปตาม ถ.ปิ่นเกล้า-นครชัยศรี เมื่อถึงพุทธมณฑลสาย 3 ก็เลี้ยวเข้าไปประมาณ 500 เมตร ร้านอยู่ทางซ้ายมือค่ะ มีคนบอกมาว่าความพิเศษของร้านนี้คือกุ๊กเก่าและเป็นกุ๊กรุ่นสุดท้ายจาก "ฮก ลก ซิ่ว” ภัตตาคารจีนชื่อดังย่านราชประสงค์ ที่เลิกกิจการไปแล้ว ต้องคนที่อายุเลข 5 ขึ้นไปจึงจะเคยกิน ฮก ลก ซิ่ว  จานเด็ดที่มีขายที่นี่แห่งเดียวในเมืองไทยคือ ปลาเต๋าเต้ย 2 ฤดู เป็นสูตรจากมาเลเซีย นอกนั้นก็มี ผัดผักน้ำมันหอย ไก่เบตง เคาหยก ปูทะเลซุปน้ำใสหม้อไฟ เต้าหู้แคระยัดไส้หม้อดิน และ ลูกชิ้นแคระ 
อาหารที่เราแนะนำคือไก่เบตง (คล้ายๆไก่แช่เหล้า) เสริฟพร้อมกับหอมเจียว และน้ำจิ้มน้ำพริกเผาสูตรเด็ดของทางร้าน 
เมนูข้าวผัดหนำเลียบก็อร่อยค่ะ ชอบมากๆ


In [10]:
word_test_df2 = pd.DataFrame(word_test)
#print(word_test_df2)

n_pad = 21
n_pad_2 = int((n_pad - 1)/2)
pad = [{0: ' '}]
df_pad = pd.DataFrame(pad * n_pad_2)
#print(df_pad)

word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
#print(word_test_df2[0][:21])
word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))

In [11]:
df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
             ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

# convert pandas dataframe to numpy array to feed to the model
x_char = df_with_context[char_row].as_matrix()

In [12]:
y_pred = my_best_model2.predict(x_char)

prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
y_pred = np.apply_along_axis(prob_to_class,1,y_pred)

In [13]:
print(len(y_pred))
print(len(word_test))
word_token = []
word_tmp = ''
for i in range(len(y_pred)):
    if(y_pred[i] == 1):
        word_token.append(word_tmp)
        word_tmp = ''
    word_tmp += word_test[i]
else:
    word_token.append(word_tmp)
word_token = word_token[1:]
print(word_token)

701
701
['อาหาร', 'ที่', 'นี่', 'เป็น', 'อาหาร', 'จีน', 'แคะ', 'ที่', 'หากิน', 'ยาก', 'ใน', 'บ้าน', 'เรา', ' ', 'ตัว', 'ร้าน', 'ตั้ง', 'อยู่', 'ที่', 'ถนนพุทธมณฑล', 'สาย', ' ', '3', ' ', 'ไป', 'ตาม', ' ', 'ถ.', 'ปิ่นเกล้า', '-', 'นครชัยศรี', ' ', 'เมื่อ', 'ถึง', 'พุทธมณฑล', 'สาย', ' ', '3', ' ', 'ก็', 'เลี้ยว', 'เข้า', 'ไป', 'ประมาณ', ' ', '500', ' ', 'เมตร', ' ', 'ร้าน', 'อยู่', 'ทาง', 'ซ้าย', 'มือ', 'ค่ะ', ' ', 'มี', 'คน', 'บอก', 'มา', 'ว่า', 'ความ', 'พิเศษ', 'ของ', 'ร้าน', 'นี้', 'คือ', 'กุ๊ก', 'เก่า', 'และ', 'เป็น', 'กุ๊ก', 'รุ่น', 'สุดท้าย', 'จาก', ' ', '"', 'ฮก', ' ', 'ลก', ' ', 'ซิ่ว', '”', ' ', 'ภัตตา', 'คาร', 'จีน', 'ชื่อ', 'ดัง', 'ย่าน', 'ราชประสงค์', ' ', 'ที่', 'เลิก', 'กิจการ', 'ไป', 'แล้ว', ' ', 'ต้อง', 'คน', 'ที่', 'อายุ', 'เลข', ' ', '5', ' ', 'ขึ้น', 'ไป', 'จึง', 'จะ', 'เคย', 'กิน', ' ', 'ฮก', ' ', 'ลก', ' ', 'ซิ่ว', ' ', ' ', 'จาน', 'เด็ด', 'ที่', 'มี', 'ขาย', 'ที่', 'นี่', 'แห่ง', 'เดียว', 'ใน', 'เมืองไทย', 'คือ', ' ', 'ปลาเต๋าเต้ย', ' ', '2', ' ', 'ฤดู', ' ', 'เป็น'

In [14]:
def cutKumKrub(string):
    word_test = [x for x in string]
    word_test_df2 = pd.DataFrame(word_test)

    n_pad = 21
    n_pad_2 = int((n_pad - 1)/2)
    pad = [{0: ' '}]
    df_pad = pd.DataFrame(pad * n_pad_2)

    word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
    word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))
    
    df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

    char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
                 ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

    # convert pandas dataframe to numpy array to feed to the model
    x_char = df_with_context[char_row].as_matrix()
    
    y_pred = my_best_model2.predict(x_char)

    prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
    y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    
    word_token = []
    word_tmp = ''
    for i in range(len(y_pred)):
        if(y_pred[i] == 1):
            word_token.append(word_tmp)
            word_tmp = ''
        word_tmp += word_test[i]
    else:
        word_token.append(word_tmp)
    word_token = word_token[1:]
    return word_token

In [15]:
def mergeLabel(review, rating):
    noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
    cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
    for i in range(len(cleaned)):
        cleaned[i] = cleaned[i].replace("\n", "")
        cleaned[i] = cleaned[i].replace("(", "")
        cleaned[i] = cleaned[i].replace(")", "")
    finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))
    label = [int(rating)] * len(cleaned)
    training_data = list(zip(cleaned, label))
    return training_data

In [16]:
mergeLabel(df[0][1], df[1][1])

[('อาหาร', 4),
 ('ที่', 4),
 ('นี่', 4),
 ('เป็น', 4),
 ('อาหาร', 4),
 ('จีน', 4),
 ('แคะ', 4),
 ('ที่', 4),
 ('หากิน', 4),
 ('ยาก', 4),
 ('ใน', 4),
 ('บ้าน', 4),
 ('เรา', 4),
 ('ตัว', 4),
 ('ร้าน', 4),
 ('ตั้ง', 4),
 ('อยู่', 4),
 ('ที่', 4),
 ('ถนนพุทธมณฑล', 4),
 ('สาย', 4),
 ('3', 4),
 ('ไป', 4),
 ('ตาม', 4),
 ('ถ.', 4),
 ('ปิ่นเกล้า', 4),
 ('-', 4),
 ('นครชัยศรี', 4),
 ('เมื่อ', 4),
 ('ถึง', 4),
 ('พุทธมณฑล', 4),
 ('สาย', 4),
 ('3', 4),
 ('ก็', 4),
 ('เลี้ยว', 4),
 ('เข้า', 4),
 ('ไป', 4),
 ('ประมาณ', 4),
 ('500', 4),
 ('เมตร', 4),
 ('ร้าน', 4),
 ('อยู่', 4),
 ('ทาง', 4),
 ('ซ้าย', 4),
 ('มือ', 4),
 ('ค่ะ', 4),
 ('มี', 4),
 ('คน', 4),
 ('บอก', 4),
 ('มา', 4),
 ('ว่า', 4),
 ('ความ', 4),
 ('พิเศษ', 4),
 ('ของ', 4),
 ('ร้าน', 4),
 ('นี้', 4),
 ('คือ', 4),
 ('กุ๊ก', 4),
 ('เก่า', 4),
 ('และ', 4),
 ('เป็น', 4),
 ('กุ๊ก', 4),
 ('รุ่น', 4),
 ('สุดท้าย', 4),
 ('จาก', 4),
 ('"', 4),
 ('ฮก', 4),
 ('ลก', 4),
 ('ซิ่ว', 4),
 ('”', 4),
 ('ภัตตา', 4),
 ('คาร', 4),
 ('จีน', 4),
 ('ชื่อ', 4),
 ('ดั

In [17]:
demo = df[:][:40000]
demo

,0,1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...,4
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...,5
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...,5


In [18]:
train = []
for i in range(len(demo)):
    train.append((demo[0][i], demo[1][i]))

In [19]:
# train

In [40]:
import re
def remove_htmlTags(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext
    
def isValidWord(text):
    dic = {text[0]:1}
    for c in text[1:]:
        if c not in dic:
            return True
        else:
            dic[c] += 1
    if dic[text[0]] > 1:
        return False
    else:
        return True
        
def tokenize(review):
    
#     tags = tltk.nlp.pos_tag(review.replace("\n", ""))
    noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
    cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
    for i in range(len(cleaned)):
        cleaned[i] = cleaned[i].replace("\n", "")
        cleaned[i] = cleaned[i].replace("(", "")
        cleaned[i] = cleaned[i].replace(")", "")
    finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))

#     # Filter parts of speech of text
#     tags = tltk.nlp.pos_tag(remove_htmlTags(review.replace("\n", "")))
#     finalWord = []
#     filteredTags = {'NOUN', 'VERB', 'ADJ','ADV', 'AUX', 'PART'}
#     for sentence in tags:
#         for word, tag in sentence:
#             if tag in filteredTags:
#                 if len(word) > 1 and isValidWord(word):
#                     finalWord.append(remove_htmlTags(word))
    return finalWord

# tokenize(train[0][0])
# train[0][0]

In [41]:
# from itertools import chain
# vocabulary = set(chain(*[tokenize(i[0].lower()) for i in train]))

In [42]:
# vocabulary
tokenize(train[999][0])

['จริง',
 'ๆ',
 'ผม',
 'ไม่',
 'รู้จัก',
 'ร้าน',
 'นี้',
 'หรอก',
 'ครับ',
 'เพราะ',
 'ไม่',
 'ค่อย',
 'ได้',
 'มา',
 'แถว',
 'นี้',
 'จน',
 'พี่',
 'สาว',
 'แฟน',
 'ชวน',
 'มา',
 'กิน',
 'ก็',
 'เปิด',
 'ดู',
 'เว็บหาแผนที่',
 'ร้าน',
 'เห็น',
 'รูปร้าน',
 'สวย',
 'มากครับ',
 'มี',
 'โต๊ะ',
 'ริม',
 'น้ำ',
 'ไม่',
 'มาก',
 'ต้อง',
 'รีบ',
 'จอง',
 'ห้องแอร์',
 'ก็',
 'มี',
 'เหมือน',
 'กัน',
 'แต่',
 'ผม',
 'ว่า',
 'ด้าน',
 'นอก',
 'บรรยากาศ',
 'ดีกว่า',
 'ครับ',
 'อา',
 'หาร',
 'ราคา',
 'ค่อนข้าง',
 'สูง',
 'แต่',
 'ปริมาณ',
 'เยอะ',
 'และ',
 'รส',
 'ชาติ',
 'ดี',
 'ครับ',
 'กุ้ง',
 'หอบ',
 'ปูปลาสดดี',
 'มี',
 'คุณภาพ',
 'พนักงานบริิการ',
 'ดี',
 'ครับ',
 'มี',
 'อารมณ์',
 'ขัน',
 'ด้วย',
 'แม้',
 'อาหาร',
 'จะ',
 'นาน',
 'ไป',
 'บ้างแต่',
 'พนักงาน',
 'ไม่',
 'ชัก',
 'สีหร้า',
 'เวลา',
 'โดน',
 'ทวง',
 'ถาม',
 'บ่อย',
 'ๆ']

In [43]:
from IPython.display import clear_output
corpus = []
for i in range(len(train)):
    print(str(i+1) + "/" + str(len(train)))
    corpus.append(tokenize(train[i][0]))

corpus

1/40000
2/40000
3/40000
4/40000
5/40000
6/40000
7/40000
8/40000
9/40000
10/40000
11/40000
12/40000
13/40000
14/40000
15/40000
16/40000
17/40000
18/40000
19/40000
20/40000
21/40000
22/40000
23/40000
24/40000
25/40000
26/40000
27/40000
28/40000
29/40000
30/40000
31/40000
32/40000
33/40000
34/40000
35/40000
36/40000
37/40000
38/40000
39/40000
40/40000
41/40000
42/40000
43/40000
44/40000
45/40000
46/40000
47/40000
48/40000
49/40000
50/40000
51/40000
52/40000
53/40000
54/40000
55/40000
56/40000
57/40000
58/40000
59/40000
60/40000
61/40000
62/40000
63/40000
64/40000
65/40000
66/40000
67/40000
68/40000
69/40000
70/40000
71/40000
72/40000
73/40000
74/40000
75/40000
76/40000
77/40000
78/40000
79/40000
80/40000
81/40000
82/40000
83/40000
84/40000
85/40000
86/40000
87/40000
88/40000
89/40000
90/40000
91/40000
92/40000
93/40000
94/40000
95/40000
96/40000
97/40000
98/40000
99/40000
100/40000
101/40000
102/40000
103/40000
104/40000
105/40000
106/40000
107/40000
108/40000
109/40000
110/40000
111/4000

833/40000
834/40000
835/40000
836/40000
837/40000
838/40000
839/40000
840/40000
841/40000
842/40000
843/40000
844/40000
845/40000
846/40000
847/40000
848/40000
849/40000
850/40000
851/40000
852/40000
853/40000
854/40000
855/40000
856/40000
857/40000
858/40000
859/40000
860/40000
861/40000
862/40000
863/40000
864/40000
865/40000
866/40000
867/40000
868/40000
869/40000
870/40000
871/40000
872/40000
873/40000
874/40000
875/40000
876/40000
877/40000
878/40000
879/40000
880/40000
881/40000
882/40000
883/40000
884/40000
885/40000
886/40000
887/40000
888/40000
889/40000
890/40000
891/40000
892/40000
893/40000
894/40000
895/40000
896/40000
897/40000
898/40000
899/40000
900/40000
901/40000
902/40000
903/40000
904/40000
905/40000
906/40000
907/40000
908/40000
909/40000
910/40000
911/40000
912/40000
913/40000
914/40000
915/40000
916/40000
917/40000
918/40000
919/40000
920/40000
921/40000
922/40000
923/40000
924/40000
925/40000
926/40000
927/40000
928/40000
929/40000
930/40000
931/40000
932/40000


1596/40000
1597/40000
1598/40000
1599/40000
1600/40000
1601/40000
1602/40000
1603/40000
1604/40000
1605/40000
1606/40000
1607/40000
1608/40000
1609/40000
1610/40000
1611/40000
1612/40000
1613/40000
1614/40000
1615/40000
1616/40000
1617/40000
1618/40000
1619/40000
1620/40000
1621/40000
1622/40000
1623/40000
1624/40000
1625/40000
1626/40000
1627/40000
1628/40000
1629/40000
1630/40000
1631/40000
1632/40000
1633/40000
1634/40000
1635/40000
1636/40000
1637/40000
1638/40000
1639/40000
1640/40000
1641/40000
1642/40000
1643/40000
1644/40000
1645/40000
1646/40000
1647/40000
1648/40000
1649/40000
1650/40000
1651/40000
1652/40000
1653/40000
1654/40000
1655/40000
1656/40000
1657/40000
1658/40000
1659/40000
1660/40000
1661/40000
1662/40000
1663/40000
1664/40000
1665/40000
1666/40000
1667/40000
1668/40000
1669/40000
1670/40000
1671/40000
1672/40000
1673/40000
1674/40000
1675/40000
1676/40000
1677/40000
1678/40000
1679/40000
1680/40000
1681/40000
1682/40000
1683/40000
1684/40000
1685/40000
1686/40000

2344/40000
2345/40000
2346/40000
2347/40000
2348/40000
2349/40000
2350/40000
2351/40000
2352/40000
2353/40000
2354/40000
2355/40000
2356/40000
2357/40000
2358/40000
2359/40000
2360/40000
2361/40000
2362/40000
2363/40000
2364/40000
2365/40000
2366/40000
2367/40000
2368/40000
2369/40000
2370/40000
2371/40000
2372/40000
2373/40000
2374/40000
2375/40000
2376/40000
2377/40000
2378/40000
2379/40000
2380/40000
2381/40000
2382/40000
2383/40000
2384/40000
2385/40000
2386/40000
2387/40000
2388/40000
2389/40000
2390/40000
2391/40000
2392/40000
2393/40000
2394/40000
2395/40000
2396/40000
2397/40000
2398/40000
2399/40000
2400/40000
2401/40000
2402/40000
2403/40000
2404/40000
2405/40000
2406/40000
2407/40000
2408/40000
2409/40000
2410/40000
2411/40000
2412/40000
2413/40000
2414/40000
2415/40000
2416/40000
2417/40000
2418/40000
2419/40000
2420/40000
2421/40000
2422/40000
2423/40000
2424/40000
2425/40000
2426/40000
2427/40000
2428/40000
2429/40000
2430/40000
2431/40000
2432/40000
2433/40000
2434/40000

3090/40000
3091/40000
3092/40000
3093/40000
3094/40000
3095/40000
3096/40000
3097/40000
3098/40000
3099/40000
3100/40000
3101/40000
3102/40000
3103/40000
3104/40000
3105/40000
3106/40000
3107/40000
3108/40000
3109/40000
3110/40000
3111/40000
3112/40000
3113/40000
3114/40000
3115/40000
3116/40000
3117/40000
3118/40000
3119/40000
3120/40000
3121/40000
3122/40000
3123/40000
3124/40000
3125/40000
3126/40000
3127/40000
3128/40000
3129/40000
3130/40000
3131/40000
3132/40000
3133/40000
3134/40000
3135/40000
3136/40000
3137/40000
3138/40000
3139/40000
3140/40000
3141/40000
3142/40000
3143/40000
3144/40000
3145/40000
3146/40000
3147/40000
3148/40000
3149/40000
3150/40000
3151/40000
3152/40000
3153/40000
3154/40000
3155/40000
3156/40000
3157/40000
3158/40000
3159/40000
3160/40000
3161/40000
3162/40000
3163/40000
3164/40000
3165/40000
3166/40000
3167/40000
3168/40000
3169/40000
3170/40000
3171/40000
3172/40000
3173/40000
3174/40000
3175/40000
3176/40000
3177/40000
3178/40000
3179/40000
3180/40000

...

37273/40000
37274/40000
37275/40000
37276/40000
37277/40000
37278/40000
37279/40000
37280/40000
37281/40000
37282/40000
37283/40000
37284/40000
37285/40000
37286/40000
37287/40000
37288/40000
37289/40000
37290/40000
37291/40000
37292/40000
37293/40000
37294/40000
37295/40000
37296/40000
37297/40000
37298/40000
37299/40000
37300/40000
37301/40000
37302/40000
37303/40000
37304/40000
37305/40000
37306/40000
37307/40000
37308/40000
37309/40000
37310/40000
37311/40000
37312/40000
37313/40000
37314/40000
37315/40000
37316/40000
37317/40000
37318/40000
37319/40000
37320/40000
37321/40000
37322/40000
37323/40000
37324/40000
37325/40000
37326/40000
37327/40000
37328/40000
37329/40000
37330/40000
37331/40000
37332/40000
37333/40000
37334/40000
37335/40000
37336/40000
37337/40000
37338/40000
37339/40000
37340/40000
37341/40000
37342/40000
37343/40000
37344/40000
37345/40000
37346/40000
37347/40000
37348/40000
37349/40000
37350/40000
37351/40000
37352/40000
37353/40000
37354/40000
37355/40000
3735

37962/40000
37963/40000
37964/40000
37965/40000
37966/40000
37967/40000
37968/40000
37969/40000
37970/40000
37971/40000
37972/40000
37973/40000
37974/40000
37975/40000
37976/40000
37977/40000
37978/40000
37979/40000
37980/40000
37981/40000
37982/40000
37983/40000
37984/40000
37985/40000
37986/40000
37987/40000
37988/40000
37989/40000
37990/40000
37991/40000
37992/40000
37993/40000
37994/40000
37995/40000
37996/40000
37997/40000
37998/40000
37999/40000
38000/40000
38001/40000
38002/40000
38003/40000
38004/40000
38005/40000
38006/40000
38007/40000
38008/40000
38009/40000
38010/40000
38011/40000
38012/40000
38013/40000
38014/40000
38015/40000
38016/40000
38017/40000
38018/40000
38019/40000
38020/40000
38021/40000
38022/40000
38023/40000
38024/40000
38025/40000
38026/40000
38027/40000
38028/40000
38029/40000
38030/40000
38031/40000
38032/40000
38033/40000
38034/40000
38035/40000
38036/40000
38037/40000
38038/40000
38039/40000
38040/40000
38041/40000
38042/40000
38043/40000
38044/40000
3804

38647/40000
38648/40000
38649/40000
38650/40000
38651/40000
38652/40000
38653/40000
38654/40000
38655/40000
38656/40000
38657/40000
38658/40000
38659/40000
38660/40000
38661/40000
38662/40000
38663/40000
38664/40000
38665/40000
38666/40000
38667/40000
38668/40000
38669/40000
38670/40000
38671/40000
38672/40000
38673/40000
38674/40000
38675/40000
38676/40000
38677/40000
38678/40000
38679/40000
38680/40000
38681/40000
38682/40000
38683/40000
38684/40000
38685/40000
38686/40000
38687/40000
38688/40000
38689/40000
38690/40000
38691/40000
38692/40000
38693/40000
38694/40000
38695/40000
38696/40000
38697/40000
38698/40000
38699/40000
38700/40000
38701/40000
38702/40000
38703/40000
38704/40000
38705/40000
38706/40000
38707/40000
38708/40000
38709/40000
38710/40000
38711/40000
38712/40000
38713/40000
38714/40000
38715/40000
38716/40000
38717/40000
38718/40000
38719/40000
38720/40000
38721/40000
38722/40000
38723/40000
38724/40000
38725/40000
38726/40000
38727/40000
38728/40000
38729/40000
3873

39336/40000
39337/40000
39338/40000
39339/40000
39340/40000
39341/40000
39342/40000
39343/40000
39344/40000
39345/40000
39346/40000
39347/40000
39348/40000
39349/40000
39350/40000
39351/40000
39352/40000
39353/40000
39354/40000
39355/40000
39356/40000
39357/40000
39358/40000
39359/40000
39360/40000
39361/40000
39362/40000
39363/40000
39364/40000
39365/40000
39366/40000
39367/40000
39368/40000
39369/40000
39370/40000
39371/40000
39372/40000
39373/40000
39374/40000
39375/40000
39376/40000
39377/40000
39378/40000
39379/40000
39380/40000
39381/40000
39382/40000
39383/40000
39384/40000
39385/40000
39386/40000
39387/40000
39388/40000
39389/40000
39390/40000
39391/40000
39392/40000
39393/40000
39394/40000
39395/40000
39396/40000
39397/40000
39398/40000
39399/40000
39400/40000
39401/40000
39402/40000
39403/40000
39404/40000
39405/40000
39406/40000
39407/40000
39408/40000
39409/40000
39410/40000
39411/40000
39412/40000
39413/40000
39414/40000
39415/40000
39416/40000
39417/40000
39418/40000
3941

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [20]:

import pickle


# pickle_out = open("corpus3.pickle","wb")
# pickle.dump(corpus, pickle_out)
# pickle_out.close()


In [21]:
pickle_in = open("corpus3.pickle","rb")
corpus = pickle.load(pickle_in)
len(corpus)
# corpus

40000

In [22]:
# corpus
# corpus = corpus[:4000]
newcor = corpus[:36000]
test = corpus[36000:]
# tltk.nlp.pos_tag(train[0][0])
len(newcor)

36000

In [23]:

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, binary = True)
tfidf_matrix = tfidf.fit_transform(newcor)

In [24]:
tfidf_matrix

<36000x158712 sparse matrix of type '<class 'numpy.float64'>'
	with 3225360 stored elements in Compressed Sparse Row format>

In [25]:

# idf = tfidf.idf_
# print (tfidf.get_feature_names(), idf)

In [26]:
# corpus = [
#     tokenize(df[0][0]),
#     tokenize(df[0][1]),
#     tokenize(df[0][2])
# ]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
# pca = PCA(n_components = 300)
x_train = tfidf_matrix.toarray()
# x_train = pca.fit_transform(x_train)

svd = TruncatedSVD(n_components=300, n_iter=7, random_state=42)
x_train = svd.fit_transform(x_train) 

y_train = []

for i in range(36000):
    y_train.append(train[i][1])
y_train = np.array(y_train)

In [ ]:
x_train.shape
# tfidf_matrix.toarray().shape

In [135]:
#Dont run
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import f1_score

classifier = SVC(kernel = 'rbf')
linearClassifier = SVC(kernel = 'linear')
# classifer = GaussianNB()
classifier.fit(x_train, y_train)
linearClassifier.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [43]:

tfidf_matrix = tfidf.transform(test)
# print(tfidf_matrix)
x_test = pca.transform(tfidf_matrix.toarray())
y_test = []

for i in range(36000,10000):
    y_test.append(train[i][1])
y_test = np.array(y_test)

# predictions = classifier.predict(x_test)


In [137]:
# x_train.mean()
# predictions
f1_score(y_test, predictions, average='macro')

/Users/ekkalakleelasornchai/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.1150997150997151

In [138]:
predictions = linearClassifier.predict(x_test)
predictions
f1_score(y_test, predictions, average='macro')

/Users/ekkalakleelasornchai/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.18891432308698497

In [44]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.layers import Dropout
from keras import metrics
from sklearn.preprocessing import LabelEncoder

from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))
rate = 0.5

def get_feedforward_nn():
  input1 = Input(shape=(300,))
  x = Dense(300, activation='relu')(input1)
  x = Dropout(rate)(x)
  x = Dense(128, activation='relu')(x)
  x = Dropout(rate)(x)  
  out = Dense(1, activation='relu')(x)

  model = Model(inputs=input1, outputs=out)
  model.compile(optimizer=Adam(),
                loss='mean_squared_error',
                metrics=[f1])
  return model

In [45]:
from keras.callbacks import ModelCheckpoint, TensorBoard

# This is called to clear the original model session in order to use TensorBoard
from keras import backend as K
K.clear_session()

# Path to save model parameters
weight_path_feedforward_nn='~/workspace/model/model_weight_feedforward_nn2.h5'

# Training callbacks list. TensorBoard() write logs for tensorboard GUI. 
# ModelCheckpoint() writes the resulting model.
# Note that writing to disk takes time (longer than model training time). 
# For other sections, you might not writing any files to disk 
# or write only the graph for TensorBoard.
callbacks_list_feedforward_nn = [
        TensorBoard(log_dir='~/workspace/model/Graph/ff2', histogram_freq=1, write_graph=True, write_grads=True),
        ModelCheckpoint(
            weight_path_feedforward_nn,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]

print(x_test.shape)
verbose = 1
model_feedforward_nn = get_feedforward_nn()
train_params = [(50, 32)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_feedforward_nn.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           callbacks=callbacks_list_feedforward_nn,
                           validation_data=(x_test, y_test))

(3000, 300)
train with 50 epochs and 32 batch size
Train on 7000 samples, validate on 3000 samples
Epoch 1/50
7000/7000 [==============================] - 1s 181us/step - loss: 2.0105 - f1: 1.0146 - val_loss: 0.6122 - val_f1: 1.0000

Epoch 00001: val_loss improved from inf to 0.61221, saving model to ~/workspace/model/model_weight_feedforward_nn2.h5
Epoch 2/50
7000/7000 [==============================] - 1s 96us/step - loss: 0.8175 - f1: 1.0000 - val_loss: 0.5995 - val_f1: 1.0000

Epoch 00002: val_loss improved from 0.61221 to 0.59947, saving model to ~/workspace/model/model_weight_feedforward_nn2.h5
Epoch 3/50
7000/7000 [==============================] - 1s 97us/step - loss: 0.7934 - f1: 1.0000 - val_loss: 0.5563 - val_f1: 1.0000

Epoch 00003: val_loss improved from 0.59947 to 0.55634, saving model to ~/workspace/model/model_weight_feedforward_nn2.h5
Epoch 4/50
7000/7000 [==============================] - 1s 96us/step - loss: 0.7570 - f1: 1.0000 - val_loss: 0.5949 - val_f1: 1.0000

Ep

In [46]:
# from keras.layers import Dropout

# def get_nn_with_dropout():
    
#     rate = 0.1
    
#     input1 = Input(shape=(21,))
#     x = Dense(100, activation='relu')(input1)
#     x = Dropout(rate)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(rate)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(rate)(x)
#     out = Dense(1, activation='sigmoid')(x)

#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                 loss='binary_crossentropy',
#                 metrics=['acc'])
#     return model

from sklearn.metrics import f1_score,precision_score,recall_score

################################################################################
# Write a function to evaluate your model. Your function must make prediction  #
# using the input model and return f-score, precision, and recall of the model.#
# You can make predictions by calling model.predict().                         #
################################################################################
def evaluate(x_test, y_test, model):
    """
    Evaluate model on the splitted 10 percent testing set.
    """
    y_pred = model.predict(x_test)

#     #map probability to class
#     prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
#     y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    for i in range(len(y_pred)):
        if y_pred[i] - int(y_pred[i]) >= 0.5:
            y_pred[i] = int(y_pred[i]) + 1
        else:
            y_pred[i] = int(y_pred[i])
    print(y_pred.T[0])
    print(y_test)
    f1score = f1_score(y_test,y_pred.T[0], average='macro')
    precision = precision_score(y_test,y_pred,average='macro')
    recall = recall_score(y_test,y_pred,average='macro')
    return f1score, precision, recall

In [47]:
# print(x_test.shape)
evaluate(x_test, y_test, model_feedforward_nn)

[ 3.  3.  3. ...,  3.  3.  4.]
[3 2 5 ..., 5 2 5]


(0.27718856039527318, 0.3155007173138572, 0.29539229047059606)